In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler  # Mixed Precision Training을 위한 임포트


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # CUDA 11.8 기준

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
# BottleneckBlock 정의
class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.dropRate > 0:
            out = nn.functional.dropout(out, p=self.dropRate, training=self.training)
        out = self.conv2(self.relu(self.bn2(out)))
        if self.dropRate > 0:
            out = nn.functional.dropout(out, p=self.dropRate, training=self.training)
        return torch.cat([x, out], 1)

# BasicBlock 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.dropRate > 0:
            out = nn.functional.dropout(out, p=self.dropRate, training=self.training)
        return torch.cat([x, out], 1)

# TransitionBlock 정의
class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.dropRate > 0:
            out = nn.functional.dropout(out, p=self.dropRate, training=self.training)
        return nn.functional.avg_pool2d(out, 2)

# DenseBlock 정의
class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(nb_layers):
            layers.append(block(in_planes + i * growth_rate, growth_rate, dropRate))
        self.layer = nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)


In [4]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import densenet121

# Google Drive 경로 설정 (데이터 경로에 맞게 수정)
train_data_path = '/content/drive/MyDrive/train'
test_data_path = '/content/drive/MyDrive/test'

# 데이터 전처리
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # 모든 이미지를 224x224로 리사이즈
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # 모든 이미지를 224x224로 리사이즈
    transforms.ToTensor(),
])

# 데이터 로드
full_train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform_train)
train_size = int(0.75 * len(full_train_dataset))  # 학습 데이터 : 검증 데이터 = 3:1
val_size = len(full_train_dataset) - train_size

# 학습 데이터와 검증 데이터로 분할
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# 테스트 데이터 로드
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform_test)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 사전 학습된 DenseNet 모델 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = densenet121(pretrained=True)  # 사전 학습된 DenseNet121 모델
num_classes = len(full_train_dataset.classes)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)  # 마지막 레이어 수정
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

# 학습 함수 정의
def train(train_loader, model, criterion, optimizer, epoch, accumulation_steps=4):
    model.train()
    optimizer.zero_grad()
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss = loss / accumulation_steps

        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % 20 == 0:
            print(f'Epoch {epoch}, Step {i}, Loss: {loss.item()}')

# 검증 함수 정의
def validate(val_loader, model, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    avg_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')
    return accuracy

# 테스트 함수 정의
def test(test_loader, model, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# 학습 및 테스트 실행
for epoch in range(30):  # 30 에포크
    train(train_loader, model, criterion, optimizer, epoch)
    val_accuracy = validate(val_loader, model, criterion)
    print(f'Epoch {epoch}, Validation Accuracy: {val_accuracy:.2f}%')

test_accuracy = test(test_loader, model, criterion)
print(f'Final Test Accuracy: {test_accuracy:.2f}%')


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 69.0MB/s]


Epoch 0, Step 0, Loss: 0.4542021155357361
Validation Loss: 1.4958, Validation Accuracy: 50.28%
Epoch 0, Validation Accuracy: 50.28%
Epoch 1, Step 0, Loss: 0.3385826647281647
Validation Loss: 0.8153, Validation Accuracy: 76.27%
Epoch 1, Validation Accuracy: 76.27%
Epoch 2, Step 0, Loss: 0.14493772387504578
Validation Loss: 0.5361, Validation Accuracy: 82.49%
Epoch 2, Validation Accuracy: 82.49%
Epoch 3, Step 0, Loss: 0.0816318690776825
Validation Loss: 0.4209, Validation Accuracy: 85.31%
Epoch 3, Validation Accuracy: 85.31%
Epoch 4, Step 0, Loss: 0.05769184231758118
Validation Loss: 0.4067, Validation Accuracy: 85.88%
Epoch 4, Validation Accuracy: 85.88%
Epoch 5, Step 0, Loss: 0.025992587208747864
Validation Loss: 0.3899, Validation Accuracy: 87.01%
Epoch 5, Validation Accuracy: 87.01%
Epoch 6, Step 0, Loss: 0.017231382429599762
Validation Loss: 0.3404, Validation Accuracy: 89.27%
Epoch 6, Validation Accuracy: 89.27%
Epoch 7, Step 0, Loss: 0.004168588202446699
Validation Loss: 0.3314, V

In [5]:
# 데이터 크기 확인용
for images, labels in train_loader:
    print(f"Image batch dimensions: {images.shape}")
    break


Image batch dimensions: torch.Size([32, 3, 224, 224])
